# 04 — Hipótese 1: DenseNet121
## Arquitetura Mais Profunda | Augmentation Leve | Sem Class Weight

Projeto: Classificação de Pneumonia em Raio-X  

Diretório raiz esperado:

C:/projects/xray-project/

Objetivo deste experimento:

Testar a hipótese de que arquiteturas mais profundas
(DenseNet121) conseguem capturar padrões mais complexos
em imagens de raio-X quando comparadas à ResNet18.

Configuração:

- Modelo: DenseNet121
- Augmentation: leve
- Class Weight: não utilizado
- Métrica principal: ROC-AUC

Arquivos utilizados:
- data/metadata/train_split.csv
- data/metadata/val_split.csv

Arquivos gerados:
- models/densenet121_light_noCW.pt
- outputs/metrics/densenet121_light_noCW.npz

Este experimento será comparado diretamente com o baseline.


In [1]:
import os

NOTEBOOK_DIR = os.getcwd()
PROJECT_ROOT = os.path.abspath(os.path.join(NOTEBOOK_DIR, ".."))

print("Project root:", PROJECT_ROOT)


Project root: c:\projects\xray-project


In [2]:
import sys
import torch
import torch.nn as nn
import pandas as pd
import numpy as np

SRC_PATH = os.path.join(PROJECT_ROOT, "src")

if SRC_PATH not in sys.path:
    sys.path.append(SRC_PATH)

from dataset import XRayDataset
from transforms import get_transforms
from model import get_model
from train_utils import train_model
from utils import set_seed, create_directories, save_metrics
from torch.utils.data import DataLoader


## Reprodutibilidade

Mantemos a mesma seed utilizada no baseline
para garantir comparação justa.


In [3]:
set_seed(42)


## Preparação de Diretórios

Garantimos a existência das pastas:

- models/
- outputs/metrics/
v

In [4]:
models_dir, metrics_dir, _ = create_directories(PROJECT_ROOT)


## Carregamento dos Splits

Utilizamos os mesmos splits congelados
para manter o controle experimental.


In [5]:
metadata_dir = os.path.join(PROJECT_ROOT, "data", "metadata")

train_df = pd.read_csv(os.path.join(metadata_dir, "train_split.csv"))
val_df = pd.read_csv(os.path.join(metadata_dir, "val_split.csv"))

print("Treino:", len(train_df))
print("Validação:", len(val_df))


Treino: 4093
Validação: 1139


## Configuração da Hipótese 1

Alteração em relação ao baseline:

Modelo: DenseNet121

Mantemos:
- Augmentation leve
- Sem class weight
- Mesmos hiperparâmetros


In [6]:
model_name = "densenet121"
augmentation = "light"
use_class_weight = False

batch_size = 32
epochs = 10
learning_rate = 1e-4

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Dispositivo:", device)


Dispositivo: cpu


## Datasets e DataLoaders

As transformações permanecem iguais ao baseline.


In [7]:
train_transform, val_transform = get_transforms(
    img_size=224,
    augmentation=augmentation
)

train_dataset = XRayDataset(train_df, transform=train_transform)
val_dataset = XRayDataset(val_df, transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


## Instanciação do DenseNet121

Modelo definido em src/model.py


In [8]:
model, target_layer = get_model(model_name=model_name)
model = model.to(device)


c:\Users\honor\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\honor\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


## Função de Perda e Otimizador

Sem utilização de class weight nesta etapa.


In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


## Padrão de Nomeação

Formato:
modelo_augmentation_classweight


In [10]:
model_filename = f"{model_name}_{augmentation}_noCW.pt"
metrics_filename = f"{model_name}_{augmentation}_noCW.npz"

model_save_path = os.path.join(models_dir, model_filename)
metrics_save_path = os.path.join(metrics_dir, metrics_filename)


## Treinamento

O loop de treino está implementado em src/train_utils.py.

Durante o treinamento:
- Barra de progresso por batch
- ROC-AUC calculada por época
- Melhor modelo salvo automaticamente


In [11]:
history, best_auc = train_model(
    model,
    train_loader,
    val_loader,
    criterion,
    optimizer,
    device,
    epochs,
    model_save_path
)


Epoch 1/10 [Treino]:   0%|          | 0/128 [00:00<?, ?it/s]

: 

## Salvamento das Métricas

As métricas serão comparadas com o baseline
no notebook final de análise.


In [ ]:
metrics_dict = {
    "train_loss": history["train_loss"],
    "val_loss": history["val_loss"],
    "val_auc": history["val_auc"],
    "best_auc": best_auc
}

save_metrics(metrics_dict, metrics_save_path)

print("Modelo salvo em:", model_save_path)
print("Métricas salvas em:", metrics_save_path)
print("Melhor AUC:", best_auc)
